<img style="float: center;" src='https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_header.png' alt="stsci_logo" width="900px"/>

# RW-DDT Deep-Dive Analysis Template Notebook

**Authors**: Taylor James Bell (ESA/AURA for STScI)<br>
**Last Updated**: August 04, 2025<br>
**jwst Pipeline Version**: 1.18.0 (Build 11.3)<br>
**Eureka! Pipeline Version**: https://github.com/kevin218/Eureka/tree/tjb_rwddt

Note that additional contextual information can be found in `README_DeepDive.md`

**Purpose**:<br/>

A collection of Eureka! Control Files (.ecf files) and a Eureka! Parameter File (.epf file) are contained within the DeepDive_setup folder provided on Box. These files are setup with generally reasonable and performant settings that should give a good first-look at the data to determine whether or not the observations were successful. Deep-dive analyses will start with _uncal.fits files downloaded from MAST and will then be processed through Stages 1--5 of Eureka!.

When doing aperture optimization or experimenting with different reduction parameters, you will have to repeat Stage 3 (and possibly Stages 4-5) for each reduction setting which can end up taking quite a while. To try to reduce the amount of human effort required to optimize your reduction settings, some amount of the aperture optimization is setup to be automatically done, and there are useful comments throughout this notebook which will help you experiment with other reduction settings that can't be automatically varied as easily.

**Data**:<br/>
This notebook assumes the uncal files have already been downloaded from MAST using the `rocky-worlds-utils/download_JWST.py` script.

**JWST pipeline version and CRDS context**:<br/>
This notebook was written for the calibration pipeline version given above and uses the context associated with this version of the JWST Calibration Pipeline. Information about this and other contexts can be found in the JWST Calibration Reference Data System (CRDS) [server]((https://jwst-crds.stsci.edu/)). If you use different pipeline
versions, please refer to the table [here](https://jwst-crds.stsci.edu/display_build_contexts/) to determine what context to use. To learn more about the differences for the pipeline, read the relevant [documentation](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline/jwst-operations-pipeline-build-information)

---

## Table of Contents
- [0. Importing the required components](#0.-Importing-the-required-components)
  - [0.1 Define your eventlabel and top directory](#0.1-Define-your-eventlabel-and-top-directory)
- [1. Stage 1](#1.-Stage-1)
- [2. Stage 2](#2.-Stage-2)
- [3. Stage 3 - Pixels to Lightcurve](#3.-Stage-3---Pixels-to-Lightcurve)
- [4. Stage 4 - Removing time-series outliers](#4.-Stage-4---Removing-time-series-outliers)
- [5. Stage 5 - Fitting the lightcurve](#5.-Stage-5---Fitting-the-lightcurve)
- [6. Comparing results for different aperture sizes](#6.-Comparing-results-for-different-aperture-sizes)
- [7. Choosing a final fiducial reduction](#7.-Choosing-a-final-fiducial-reduction)

---

## 0. Importing the required components

There should be no need to change any of this

In [ ]:
# Importing a bunch of Eureka! components
import eureka.lib.plots
from eureka.lib.manageevent import findevent
from eureka.lib.readECF import MetaClass
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
import eureka.S5_lightcurve_fitting.s5_fit as s5

# Set up some parameters to make plots look nicer. You can set usetex=True if you have LaTeX installed
eureka.lib.plots.set_rc(style='eureka', usetex=False, filetype='.png')

# Some imports to interact with outputs within the Jupyter notebook
from IPython.display import Image, display
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
plt.ioff()

### 0.1 Define your eventlabel and top directory

Next, we need to choose a short, meaningful label (without spaces) that describes the data we're currently working on. This eventlabel will determine will give nicknames to all your output folders and files.

We also need to tell the notebook where all our data is going to be stored

In [ ]:
# Enter in a custom eventlabel that will be used to distinguish the outputs
# from all subsequent processing
eventlabel = ''

# Specify here the top directory that will contain all ingested and output files
topdir = '/home/rwddt' ## <- ENTER YOUR TOPDIR HERE (leave at /home/rwddt if using Docker)

# Specify your analysis name here (analysis if using Docker, Analysis_A or Analysis_B otherwise)
analysis_name = 'analysis'

# A full Eureka! run through all five stages with many different aperture+annulus pairs can take many hours.
# If we want to just grab the results from previous runs and carry out comparisons, you can set process=False
# below to skip the Eureka! processing portions of the notebook.
process = True

---

## 1. Stage 1

### 1.1 Setting up the Stage 1 ECF

A good starting point for your deep-dive analysis has already been populated within the template Stage 1 ECF file, so first we'll load that in and look at the default settings

Parameters that might be worth varying are as documented in the `README_DeepDive.md` file.

In [ ]:
s1_ecf_contents = f"""# Eureka! Control File for Stage 1: Detector Processing

# Stage 1 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-1

suffix                    uncal
pmap                      1364         # Setting a fixed pmap value to ensure consistency in reductions

maximum_cores             'half'       # Options are 'none', 'quarter', 'half', 'all', or any integer

# Pipeline stages
skip_emicorr              False        #### FINDME: Might be worth turning on/off
skip_saturation           False
skip_firstframe           False        #### FINDME: Might be worth turning on/off
skip_lastframe            False        #### FINDME: Might be worth turning on/off
skip_reset                False
skip_linearity            False
skip_rscd                 False        #### FINDME: Might be worth turning on/off
skip_dark_current         False        #### FINDME: Might be worth turning on/off
skip_refpix               False        #### FINDME: Might be worth turning on/off
skip_jump                 False
skip_clean_flicker_noise  True         #### FINDME: Might be worth turning on/off

#Pipeline stages parameters
jump_rejection_threshold  8.0          #### FINDME: Might be worth trying different values

# Diagnostics
isplots_S1                3
nplots                    5
hide_plots                True
verbose                   True

# Project directory
topdir              {topdir}

# Directories relative to topdir
inputdir            Uncalibrated
outputdir           {analysis_name}/DeepDive/Stage1
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S1_{eventlabel}.ecf', 'w') as f:
    f.write(s1_ecf_contents)

### 1.2 Running Stage 1

Here we run the Eureka! Stage 1 pipeline using the settings we defined above. This should take several minutes (~11 minutes for a TRAPPIST-1b eclipse observation on a laptop with an M3 Max CPU), but that will depend on the data volume of the observation you're working on and the specifics of your CPU

If you previously ran Stage 1 and want to re-use those outputs, you can just comment-out the following line

In [ ]:
if process:
    s1_meta = s1.rampfitJWST(eventlabel)
else:
    # If not running S1, automatically grab the latest MetaData
    temp_meta = MetaClass()
    temp_meta.eventlabel = eventlabel
    temp_meta.topdir = topdir
    temp_meta.inputdir = f'{topdir}/{analysis_name}/DeepDive/Stage1/'
    s1_meta, *_ = findevent(temp_meta, 'S1')

---

## 2. Stage 2

### 2.1 Setting up the Stage 2 ECF

For deep-dive lightcurve analyses, there is no need to change any of the Stage 2 settings beyond what is provided in the template, so we'll just read those defaults in.

By default, the most recently created Stage 1 output located in `s2_meta.inputdir` will be used as the input for your Stage 2. If you instead want to use a specific Stage 1 output, you'll need to update your `inputdir` setting to point to the specific Stage 1 output folder you want to work on (e.g., `'Stage1/S1_2025-02-28_trappist1b_eclipse4_run1').

In [ ]:
s2_ecf_contents = f"""# Eureka! Control File for Stage 2: Data Reduction

# Stage 2 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-2

pmap				1364

skip_flat_field     False       #### FINDME: Might be worth turning on/off
skip_photom         True        # Strongly recommended to skip (unless doing flux calibrated photometry) in order to get better uncertainties out of Stage 3.

# Project directory
topdir              {topdir}

# Directories relative to topdir
inputdir            {analysis_name}/DeepDive/Stage1
outputdir           {analysis_name}/DeepDive/Stage2
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S2_{eventlabel}.ecf', 'w') as f:
    f.write(s2_ecf_contents)

### 2.2 Running Stage 2

Here we run the Eureka! Stage 2 pipeline using the settings we defined above. This should take <1 minute, but that will depend on the data volume of the observation you're working on and the specifics of your CPU

If you previously ran Stage 2 and want to re-use those outputs, you can just comment-out the following line

In [ ]:
if process:
    s2_meta = s2.calibrateJWST(eventlabel)
else:
    # If not running S2, automatically grab the latest MetaData
    temp_meta = MetaClass()
    temp_meta.eventlabel = eventlabel
    temp_meta.topdir = topdir
    temp_meta.inputdir = f'{topdir}/{analysis_name}/DeepDive/Stage2/'
    s2_meta, *_ = findevent(temp_meta, 'S2')

---

## 3. Stage 3 - Pixels to Lightcurve

Stage 3 performs background subtraction and optimal spectral extraction. This will generate time series of 1D spectra.

### 3.1 Setting up the Stage 3 ECF

Parameters that might be worth varying are documented in the `README_DeepDive.md` file.

By default, the most recently created Stage 2 output located in `s3_meta.inputdir` will be used as the input for your Stage 3. If you instead want to use a specific Stage 2 output, you'll need to update your `inputdir` setting to point to the specific Stage 2 output folder you want to work on (e.g., `'Stage2/S2_2025-02-28_trappist1b_eclipse4_run1').

In [ ]:
s3_ecf_contents = f"""# Eureka! Control File for Stage 3: Data Reduction

# Stage 3 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-3

ncpu            16
nfiles          200
max_memory      1.5
indep_batches   False       # Independently treat each batch of files? Strongly recommended to leave this as False unless you have a clear reason to set it to True.
suffix          calints

calibrated_spectra  False   # Set True to generate flux-calibrated spectra/photometry in mJy
                            # Set False to convert to electrons
pmap            1364

# Subarray region of interest
ywindow         None	    # Vertical axis as seen in DS9
xwindow         None        # Horizontal axis as seen in DS9
dqmask          True

# Background parameters
ff_outlier      True        # Set False to use only background region (recommended for deep transits)
                            # Set True to use full frame (works well for shallow transits/eclipses)
bg_thresh       [5,5]
interp_method   linear      # Interpolate bad pixels. Options: None (if no interpolation should be performed), linear, nearest, cubic

# Centroiding parameters
centroid_method mgmc        # Method used for centroiding. Options: mgmc, fgc
ctr_guess		fits    	# Initial guess of centroid position. If None, will first perform centroiding on whole frame (can sometimes fail)
ctr_cutout_size 10          # Cutoff size all around the centroid after the coarse centroid calculation or first centroid guess when using the mgmc method.
centroid_tech   com         # (mgmc method param) Technique used for centroiding. Options: com, 1dg, 2dg
gauss_frame     15          # (mgmc method param) Half-width away from second centroid guess to include in centroiding map for gaussian widths. Recommend ~15 for MIRI photometry.

# Photometric extraction parameters
phot_method     photutils   # photutils (aperture photometry using photutils), poet (aperture photometry using code from POET), or optimal (for optimal photometric extraction)
aperture_edge   exact       # center (pixel is included only if its center lies within the aperture), or exact (pixel is weighted by the fractional area that lies within the aperture)
aperture_shape  circle      # If phot_method is photutils or optimal: circle, ellipse, or rectangle. If phot_utils is poet: circle or hexagon. Used to set both the object aperture shape and the sky annulus shape
moving_centroid False       # Boolean: False if the aperture should stay fixed on the median centroid location (recommended), or True if the aperture should track the moving centroid
skip_apphot_bg  False       # Skips the background subtraction during the aperture photometry step
photap          [4,11,1]    # Size of photometry aperture radius in pixels
skyin           [14,26,4]   # Inner sky annulus edge, in pixels
skywidth        [10,30,10]  # Width of the sky annulus, in pixels

# Diagnostics
isplots_S3      3
nplots          3
hide_plots      False
save_output     True
save_fluxdata   False
verbose         True

# Project directory
topdir          {topdir}

# Directories relative to topdir
inputdir        {analysis_name}/DeepDive/Stage2
outputdir       {analysis_name}/DeepDive/Stage3
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S3_{eventlabel}.ecf', 'w') as f:
    f.write(s3_ecf_contents)

### 3.2 Running Stage 3

Here we run the Eureka! Stage 3 pipeline using the settings we defined above. For each aperture-annulus pair, this should take ~1 minute, but that will depend on the data volume of the observation you're working on and the specifics of your CPU.

Given that we're considering many such pairs, **this will take a substantial amount of time**

Unlike in the quick-look analysis, we're going to show the output plots as they're made (by having set s3_meta.hide_plots to False and having removed some of the Jupyter-notebook specific tweaks in the quick-look notebook). This might look a bit more cluttered, but it can be useful to examine each of the output figures as they're made to get a better understanding of what is happening and how your choices impact the reduction.

If you previously ran Stage 3 and want to re-use those outputs, you can just comment-out the following line

In [ ]:
if process:
    spec, s3_meta = s3.reduce(eventlabel)
else:
    # If not running S3, automatically grab the latest MetaData and SpecData
    temp_meta = MetaClass()
    temp_meta.eventlabel = eventlabel
    temp_meta.topdir = topdir
    temp_meta.inputdir = f'{topdir}/{analysis_name}/DeepDive/Stage3/'
    s3_meta, *_ = findevent(temp_meta, 'S3')
    spec = xr.load_dataset(s3_meta.filename_S3_SpecData)

### 3.3 Investigating the Stage 3 outputs

If there are issues with your analyses that you are having troubles working out, try re-running and increasing your `isplots_S3` setting to `5` to get more output plots that can sometimes be useful for debugging and/or increasing `nplots` to a larger number to investigate some of the repeated plots for more integrations or try setting `nplots` to `None` to make the repeated plots for all integrations (this will take a while and will make a *lot* of figures).

Generally speaking, the lower the Stage 3 MAD (median-absolute deviation) value, the better the overall reduction is since the S3 MAD is approximately what the overall noise level is in the lightcurve. This is not always the case, as sometimes lightcurves with higher MAD values have cleaner correlated noise properties that can be better decorrelated in Stage 5. But generally speaking, the lower the S3 MAD value the better so long as that isn't coming at the cost of large amounts of data points being discarded. A decent first approach is to try different Stage 3 settings until you find a "sweet-spot", and then if you want to double-check your work and conclusions, you can run Stages 4-5 on different Stage 3 outputs to confirm that your lower-MAD reduction resulted in a "better" final result.

---

## 4. Stage 4 - Removing time-series outliers

Stage 4 for photometric data just clips outliers with respect to a box-car smoothed version of the signal - this helps remove any remaining outliers while also trying to avoid removing astrophysical signals.

### 4.1 Setting up the Stage 4 ECF

Parameters that might be worth varying are documented in the `README_DeepDive.md` file.

By default, the most recently created Stage 3 output located in `s4_meta.inputdir` will be used as the input for your Stage 4. If you instead want to use a specific Stage 3 output, you'll need to update your `inputdir` setting to point to the specific Stage 3 output folder you want to work on (e.g., `'Stage3/S3_2025-02-28_trappist1b_eclipse4_run1'`).

In [ ]:
s4_ecf_contents = f"""# Eureka! Control File for Stage 4: Generate Lightcurves

# Stage 4 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-4

# Number of spectroscopic channels spread evenly over given wavelength range
nspecchan       1
compute_white   False

allapers        True    # Run S4 on all of the apertures considered in S3? Otherwise will use newest output in the inputdir

# Parameters for sigma clipping
clip_binned     True    # Whether or not sigma clipping should be performed on the binned 1D time series
sigma           3.5     # The number of sigmas a point must be from the rolling median to be considered an outlier
box_width       20      # The width of the box-car filter (used to calculated the rolling median) in units of number of data points
maxiters        20      # The number of iterations of sigma clipping that should be performed.
boundary        fill    # Use 'fill' to extend the boundary values by the median of all data points (recommended), 'wrap' to use a periodic boundary, or 'extend' to use the first/last data points
fill_value      mask    # Either the string 'mask' to mask the outlier values (recommended), 'boxcar' to replace data with the mean from the box-car filter, or a constant float-type fill value.

# Project directory
topdir          {topdir}

# Directories relative to topdir
inputdir        {analysis_name}/DeepDive/Stage3
outputdir       {analysis_name}/DeepDive/Stage4
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S4_{eventlabel}.ecf', 'w') as f:
    f.write(s4_ecf_contents)

### 4.2 Running Stage 4

Here we run the Eureka! Stage 4 pipeline using the settings we defined above. For each aperture-annulus pair, this should take << 1 minute

In [ ]:
if process:
    spec, lc, s4_meta = s4.genlc(eventlabel)
else:
    # If not running S4, automatically grab the latest MetaData and SpecData
    temp_meta = MetaClass()
    temp_meta.eventlabel = eventlabel
    temp_meta.topdir = topdir
    temp_meta.inputdir = f'{topdir}/{analysis_name}/DeepDive/Stage4/'
    s4_meta, path, _ = findevent(temp_meta, 'S4')
    spec = xr.load_dataset(s4_meta.filename_S4_SpecData)
    lc = xr.load_dataset(s4_meta.filename_S4_LCData)

### 4.3 Investigating the Stage 4 outputs

If there are still substantial outliers (think >5 sigma, since the size of the plotted error bars is likely a bit underestimated), you will need to adjust the `sigma` and `box_width` settings in your Stage 4 ECF in order to properly catch all the outliers. This might take a bit of guess-and-check work. Don't worry as much about outliers that are right at the very start of the obsevations though, as these will be removed in Stage 5 anyway when we're removing the worst of the detector settling effects

---

## 5. Stage 5 - Fitting the lightcurve

### 5.1 Setting up the Stage 5 ECF

Parameters that might be worth varying are documented in the `README_DeepDive.md` file. In particular, read the recommendations about the `allapers` setting at the top of the "Context behind the Stage 5 ECF" section.

By default, the most recently created Stage 4 output located in `s5_meta.inputdir` will be used as the input for your Stage 5. If you instead want to use a specific Stage 4 output, you'll need to update your `inputdir` setting to point to the specific Stage 4 output folder you want to work on (e.g., `'Stage4/S4_2025-02-28_trappist1b_eclipse4_run1'`).

In [ ]:
s5_ecf_contents = f"""# Eureka! Control File for Stage 5: Lightcurve Fitting

# Stage 5 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-5

ncpu            16    # The number of CPU threads to use when running emcee or dynesty in parallel

allapers        True  # Run S5 on all of the apertures considered in S4? Otherwise will use newest output in the inputdir

fit_par         ./S5_{eventlabel}.epf
fit_method      [dynesty]
run_myfuncs     [batman_ecl, polynomial, expramp, xpos, ypos, xwidth, ywidth, GP]

#GP inputs
kernel_inputs   ['time']  # options: time
kernel_class    ['Matern32']  # options: ExpSquared, Matern32, Exp, RationalQuadratic for george, Matern32 for celerite (sums of kernels possible for george separated by commas)
GP_package      'celerite'  # options: george, celerite

# Manual clipping in time
manual_clip     [[None,50]]   # Remove the first ~50 integrations which will be most affected by detector settling

# dynesty fitting parameters
run_nlive       'min'         # Must be > ndim * (ndim + 1) // 2. Use 'min' to use the minimum safe number
run_bound       'multi'
run_sample      'rwalk'
run_tol         0.01

# Plotting controls
interp          True    # Should astrophysical model be interpolated (useful for uneven sampling like that from HST)

# Diagnostics
isplots_S5      5       # Generate few (1), some (3), or many (5) figures (Options: 1 - 5)

# Project directory
topdir          {topdir}

# Directories relative to topdir
inputdir        {analysis_name}/DeepDive/Stage4
outputdir       {analysis_name}/DeepDive/Stage5
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S5_{eventlabel}.ecf', 'w') as f:
    f.write(s5_ecf_contents)

### 5.2 Setting up the Stage 5 EPF

A description of how you should adjust your ECF parameters is in the `README_DeepDive.md` file.

Make sure to update the astrophysical parameter priors based on those provided in the relevant Jira ticket

In [ ]:
s5_epf = """
# Stage 5 Fit Parameters Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-5-fit-parameters

#Name         Value                 Free?            PriorPar1        PriorPar2    PriorType
# "Free?" can be free, fixed, white_free, white_fixed, shared, or independent
# PriorType can be U (Uniform), LU (Log Uniform), or N (Normal).
# If U/LU, PriorPar1 and PriorPar2 represent lower and upper limits of the parameter/log(the parameter).
# If N, PriorPar1 is the mean and PriorPar2 is the standard deviation of a Gaussian prior.
#-------------------------------------------------------------------------------------------------------
rp          YourRadiusHere      'fixed'
fp          YourFpHere          'free'           YourFpHere    2000e-6     N
# ------------------
# Orbital parameters
# ------------------
per         YourPerHere         'fixed'
t_secondary YourTsecHere        'free'           YourTsecHere  YourTsecUncertHere N
inc         YourIncHere         'fixed'
a           YourAHere           'fixed'
ecc         0.                  'fixed'
w           90.                 'fixed'
# The following two lines are commented out, but you can uncomment them (while commenting out the ecc and w lines above) and edit them if needed for your planet
# ecosw       YourEcoswHere       'fixed'          YourEcoswHere YourEcoswUncertHere N
# esinw       YourEsinwHere       'fixed'          YourEsinwHere YourEsinwUncertHere N
# --------------------------------------------------------------------------
# Systematic variables (these can be left as-is for the Quick-Look analysis)
# --------------------------------------------------------------------------
# Polynomial Parameters
c0          0.999               'free'           0.999         0.01        N
c1          -0.002              'free'           0.0           0.1         N
# Ramp Parameters
r0          0.002               'free'           0.0           0.01        N
r1          50                  'free'           3             300         U
# Centroid decorrelation parameters
ypos        0.0                 'free'           0.0           0.5         N
xpos        0.0                 'free'           0.0           0.5         N
ywidth      0.0                 'free'           0.0           0.5         N
xwidth      0.0                 'free'           0.0           0.5         N
# Gaussian Process parameters (only used if GP added to s5_meta.run_myfuncs and uncommented below)
A           -17                 'free'          -24            -10         U
m           -4                  'free'          -7             0           U
# White noise
scatter_mult 1.4                'free'           0.8           10          U
"""

# This code will write your EPF settings to a file and doesn't need to be changed
epf_filename = f'./S5_{eventlabel}.epf'
with open(epf_filename, 'w') as file:
    file.writelines(s5_epf)

### 5.3 Running Stage 5

Here we run the Eureka! Stage 5 pipeline using the settings we defined above. This should take ~1 minute or less per fit, but that will depend on the data volume of the observation you're working on, the specifics of your CPU, and how well the model matches your data.

Again, if you've set `allapers` to `True`, then this stage will take **quite a long time** in total.

Note, if you re-run Stage 5 without restarting the Jupyter notebook, the color of your plot will change each time you run Stage 5. This is not an issue and there's no need to restart the notebook; it is just a consequence of the fact that Eureka! was not originally intended to be run within Jupyter notebooks

In [ ]:
if process:
    s5_meta = s5.fitlc(eventlabel)
else:
    # If not running S5, automatically grab the latest MetaData
    temp_meta = MetaClass()
    temp_meta.eventlabel = eventlabel
    temp_meta.topdir = topdir
    temp_meta.inputdir = f'{topdir}/{analysis_name}/DeepDive/Stage5/'
    s5_meta, *_ = findevent(temp_meta, 'S5')

---
Only proceed to the cells below once you've run your suite of Stage 5 fits with `allapers` set to `True`

---

## 6. Comparing results for different aperture sizes

### 6.1 Choosing a initial "fiducial"/"best" reduction

To start, the following plotting code will put a bit of focus on the reduction which produces the lowest eclipse-depth uncertainty. This is often a fairly reasonable choice, but make sure to use the following plots to investigate that choice and look for any potential biases or items of concern

In [ ]:
# Get the fitted eclipse depth uncertainty for every considered aperture+annulus pairing
aps = []
bgs = []
eclipseDepthError = []
for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        aps.append(ap)
        bgs.append(bg)
        eclipseDepthError.append(np.std(samples.fp.values)*1e6)
aps = np.array(aps)
bgs = np.array(bgs)
eclipseDepthError = np.array(eclipseDepthError)

best_ap = aps[np.argmin(eclipseDepthError)]
best_bg = bgs[np.argmin(eclipseDepthError)]

print(f'The automatically recommended fiducial reduction aperture+annulus pair is: ap{best_ap}_bg{best_bg}')
print('Be sure not to blindly trust this recommendation and investigate the plots below!')

#### Pulling-up the S5 plots for the initially recommended aperture+annulus pair

In [ ]:
figures = np.sort(glob(f'{s5_meta.outputdir}../ap{best_ap}_bg{best_bg}/figs/*'))
for figure in figures:
    if 'startingpoint' in figure.lower():
        continue
    display(Image(filename=figure, embed=True, width=700))

Do the above plots all look good to you? Does the model do a good job of fitting the data, does the residual lightcurve look quite flat and centered at 0 ppm, does the GP model (if fitted) look smooth and not like it was trying to fit super high-frequency white noise, does the empirical line on the Allan variance plot look quite close to the predicted trend, does the histogram of residuals look Gaussian with the same approximate standard deviation as expected, and does the corner plot look fairly converged and does it show any concerning correlations between the fitted eclipse depth or eclipse timing and any of the systematic noise parameters?

### 6.2 Comparing lightcurve noise levels vs aperture properties (rough gauge for "best" reduction)

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        if ap == s3_meta.spec_hw_range[0] and bg == s3_meta.bg_hw_range[0]:
            label2 = 'S4 MAD'
            label3 = 'S5 Scatter'
        else:
            label2 = None
            label3 = None

        meta_s4_temp = eureka.lib.manageevent.loadevent(f'{s4_meta.outputdir}../ap{ap}_bg{bg}/S4_{eventlabel}_Meta_Save.dat')
        ax.plot(ap, meta_s4_temp.mad_s4, '*', c='C1', label=label2)
        
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        fit = pd.read_csv(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_{eventlabel}_ap{ap}_bg{bg}_Table_Save_ch0.txt', delim_whitespace=True, comment='#')
        ax.errorbar(ap, np.nanmedian(fit.lcerr)*1e6, np.nanmedian(fit.lcerr)*1e6/np.median(samples.scatter_mult)*np.std(samples.scatter_mult), fmt='.', color='C2', label=label3)
        
ax.set_ylabel('Noise Level (ppm)')
ax.set_xlabel('Aperture Radius (px)')
ax.legend(loc='best')
plt.show(fig)

Is there any obvious preference for a particular aperture radius?

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        if ap == s3_meta.spec_hw_range[0] and bg == s3_meta.bg_hw_range[0]:
            label2 = 'S4 MAD'
            label3 = 'S5 Scatter'
        else:
            label2 = None
            label3 = None

        meta_s4_temp = eureka.lib.manageevent.loadevent(f'{s4_meta.outputdir}../ap{ap}_bg{bg}/S4_{eventlabel}_Meta_Save.dat')
        ax.plot(bg.split('_')[0], meta_s4_temp.mad_s4, '*', c='C1', label=label2)
        
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        fit = pd.read_csv(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_{eventlabel}_ap{ap}_bg{bg}_Table_Save_ch0.txt', delim_whitespace=True, comment='#')
        ax.errorbar(bg.split('_')[0], np.nanmedian(fit.lcerr)*1e6, np.nanmedian(fit.lcerr)*1e6/np.median(samples.scatter_mult)*np.std(samples.scatter_mult), fmt='.', color='C2', label=label3)
        
ax.set_ylabel('Noise Level (ppm)')
ax.set_xlabel('Inner Annulus Radius (px)')
ax.legend(loc='best')
plt.show(fig)

Is there any obvious preference toward a certain annulus inner radius?

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        if ap == s3_meta.spec_hw_range[0] and bg == s3_meta.bg_hw_range[0]:
            label2 = 'S4 MAD'
            label3 = 'S5 Scatter'
        else:
            label2 = None
            label3 = None

        meta_s4_temp = eureka.lib.manageevent.loadevent(f'{s4_meta.outputdir}../ap{ap}_bg{bg}/S4_{eventlabel}_Meta_Save.dat')
        ax.plot(np.diff(np.array(bg.split('_')).astype(int)), meta_s4_temp.mad_s4, '*', c='C1', label=label2)
        
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        fit = pd.read_csv(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_{eventlabel}_ap{ap}_bg{bg}_Table_Save_ch0.txt', delim_whitespace=True, comment='#')
        ax.errorbar(np.diff(np.array(bg.split('_')).astype(int)), np.nanmedian(fit.lcerr)*1e6, np.nanmedian(fit.lcerr)*1e6/np.median(samples.scatter_mult)*np.std(samples.scatter_mult), fmt='.', color='C2', label=label3)
        
ax.set_ylabel('Noise Level (ppm)')
ax.set_xlabel('Annulus Width (px)')
ax.legend(loc='best')
plt.show(fig)

Is there any obvious preference toward a certain annulus width?

### 6.3 Comparing fitted eclipse depth uncertainties vs aperture properties (a slightly better, but slightly riskier, gauge for "best" reduction)

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        ax.errorbar(ap, np.std(samples.fp)*1e6, fmt='.', color='k')

ax.set_ylabel('Fitted Eclipse Depth Uncertainty (ppm)')
ax.set_xlabel('Aperture Radius (px)')
plt.show(fig)

Is there any obvious preference for a particular aperture radius?

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        ax.errorbar(bg.split('_')[0], np.std(samples.fp)*1e6, fmt='.', color='k')

ax.set_ylabel('Fitted Eclipse Depth Uncertainty (ppm)')
ax.set_xlabel('Inner Annulus Radius (px)')
plt.show(fig)

Is there any obvious preference toward a certain annulus inner radius?

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in [best_ap,]:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        ax.errorbar(np.diff(np.array(bg.split('_')).astype(int)), np.std(samples.fp)*1e6, fmt='.', color='k')

ax.set_ylabel('Fitted Eclipse Depth Uncertainty (ppm)')
ax.set_xlabel('Annulus Width (px)')
plt.show(fig)

Is there any obvious preference toward a certain annulus width?

### 6.4 Comparing fitted eclipse depth vs aperture properties (looking for possible biases)

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        ax.errorbar(ap, np.median(samples.fp)*1e6, np.std(samples.fp)*1e6, fmt='.', color='k')

ax.set_ylabel('Fitted Eclipse Depth (ppm)')
ax.set_xlabel('Aperture Radius (px)')
plt.show(fig)

Does the above plot suggest there is any obvious bias caused by the choice of aperture radius?

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        ax.errorbar(bg.split('_')[0], np.median(samples.fp)*1e6, np.std(samples.fp)*1e6, fmt='.', color='k')

ax.set_ylabel('Fitted Eclipse Depth (ppm)')
ax.set_xlabel('Inner Annulus Radius (px)')
plt.show(fig)

Does the above plot suggest there is any obvious bias caused by the choice of inner radius of the background annulus?

In [ ]:
fig, ax = plt.subplots(1, 1, num=0)

for ap in s3_meta.spec_hw_range:
    for bg in s3_meta.bg_hw_range:
        samples = xr.load_dataset(f'{s5_meta.outputdir}../ap{ap}_bg{bg}/S5_dynesty_samples_ch0.h5', engine='h5netcdf')
        ax.errorbar(np.diff(np.array(bg.split('_')).astype(int)), np.median(samples.fp)*1e6, np.std(samples.fp)*1e6, fmt='.', color='k')

ax.set_ylabel('Fitted Eclipse Depth (ppm)')
ax.set_xlabel('Annulus Width (px)')
plt.show(fig)

Does the above plot suggest there is any obvious bias caused by the choice of the width of the background annulus?

## 7. Choosing a final fiducial reduction

If you feel the need to change the "fiducial" aperture+annulus radii from the initially suggested values, you can do any extra work needed in the following cells to work out what you think should be used instead.